In [1]:
!pip install dfply
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser
import re
import math
from dfply import *

In [2]:
transfers_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/Transfers_Full.csv"
transfers = pd.read_csv(transfers_path, index_col = 0)

mval_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/markval.csv"
markval = pd.read_csv(mval_path, index_col = 0)

players_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/Players_Full.csv"
players = pd.read_csv(players_path, index_col = 0)


def remove_chars(string):
  return string.strip()

def clean_money(string):
  ammount = string
  try:
    if("free" in string.lower()):
      ammount = 0.0
    elif("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def is_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return result

def no_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return not(result)

def clean_loans(string):
  ammount = 0.0
  try:
    if("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def call_nan(string):
  return math.isnan(string)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def clean_transfers(transfers):
  transfers = transfers.drop_duplicates()
  transfers["Name"] = transfers["Name"].apply(remove_chars)
  transfers["From"] = transfers["From"].apply(remove_chars)
  transfers["To"] = transfers["To"].apply(remove_chars)
  transfers = transfers[transfers["Player_Id"] != "\nAtalanta "]
  transfers.Player_Id = transfers.Player_Id.astype(int)
  transfers.tm_Id = transfers.tm_Id.astype(int)

  loans = transfers[transfers["Fee"].apply(is_loan)]
  transfers = transfers[(transfers["Fee"].apply(no_loan))]
  transfers["Market Value"] = transfers["Market Value"].apply(clean_money)
  transfers["Market Value"] = pd.to_numeric(transfers["Market Value"], errors="coerce")
  transfers["Fee"] = transfers["Fee"].apply(clean_money)
  transfers["Fee"] = pd.to_numeric(transfers["Fee"], errors="coerce")
  transfers["Loan"] = False

  loans["Market Value"] = loans["Market Value"].apply(clean_money)
  loans["Market Value"] = pd.to_numeric(loans["Market Value"], errors="coerce")
  loans["Fee"] = loans["Fee"].apply(clean_money)
  loans["Fee"] = pd.to_numeric(loans["Fee"], errors="coerce")
  loans["Loan"] = True
  loans.fillna(value = 0, inplace=True)

  transfers = transfers.append(loans)
  transfers["Season"] = transfers["Season"].astype('category')

  transfers.columns = ['Player_Id', 'tm_Id', 'Name', 'From', 'To', 'Fee', 'MV',
       'Season', 'Date', 'Loan']

  transfers = (transfers >>
             arrange(X.Player_Id) >>
             mask(X.Fee.notna()) >>
             drop(X.Player_Id))
  transfers.Season = transfers.Season.astype("category")
  transfers["Date"] = pd.to_datetime(transfers["Date"], errors="coerce")

  # hdf_path = "/content/drive/My Drive/Capstone/Capstone_Data/clean_data/data.h5"
  # transfers.to_hdf(hdf_path, key = "transfers", mode='w', format = "table")
  transfers.to_csv("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/Transfers_Clean.csv")
  transfers.to_pickle("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/Transfers_Clean.pkl")


In [0]:
def clean_markval(markval):
  unique_ids = list(players.tm_Id.unique())
  markval = markval[markval.tm_Id.isin(unique_ids)]
  markval.columns = ["Name", "Club", "League", "Season", "MV", "tm_Id"]
  markval = ( markval >>
              mask(X.MV.notna()) >>
              drop(X.Name) 
              )
  markval = ( markval >>
            select(X.tm_Id, X.Club, X.League, X.Season, X.MV))
  markval[markval.tm_Id == 17259]
  markval["Club"] = markval["Club"].apply(remove_chars)
  markval["League"] = markval["League"].apply(remove_chars)
  markval["MV"] = markval["MV"].apply(clean_money)
  markval["MV"] = pd.to_numeric(markval["MV"], errors="coerce")
  markval.Season = markval.Season.astype("category")
  markval.to_csv("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/markval_Clean.csv")
  markval.to_pickle("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/markval_Clean.pkl")


In [5]:
clean_transfers(transfers)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [0]:
clean_markval(markval)